___

# &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;                        Projeto 2 - Ciência dos Dados
### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;                 Alexandre Cury, Guilherme Lotaif

<br>

## 1. Problema: Classificador automático de sentimento
<br>

O Classificador que este projeto terá como base é o Naive-Bayes, que essencialmente irá determinar se um pedaço de texto é relevante ou não a um assunto especifico. Iremos trabalhar com posts coletados da rede social Twitter, em forma de tweets, estes serão divididos em dois grupos: Treinamento e teste, para que possamos fazer com que nosso classificador apreenda como discernir o conteudo dos tweets.

___

## Preparando o ambiente no jupyter:

Para fazer uma análise e classificação dos nossos dados, precisamos primeiro coletar os tweets do assunto escolhido em um arquivo excel, que será usado como nossa base de dados.

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

**Importando as bibliotecas necessárias:**

In [2]:
from random import shuffle
import pandas as pd
import os.path
import tweepy
import math
import json
import sys
import warnings

### Autenticando no  Twitter

Tendo em vista que precisamos da permissão do twitter para acessar e coletar os tweets, é necessário ter uma conta de desenvolvedor, além de solicitar as chaves de acesso. Para conseguir estas, acompanhe o [tutorial](https://developer.twitter.com/en/docs/basics/authentication/guides/access-tokens.html) o próprio twitter.

In [ ]:
#Dados de autenticação do twitter:
#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

### Escolha do produto e coleta das mensagens

Agora é feita a seleção do produto que iremos análisar, e para coletar os tweets para o nosso projeto, é preciso especificar a quantidade de posts que serão importados `(quanto maior esta quantidade for, mais preciso será o nosso classificador)`.

In [ ]:
#Produto escolhido:
produto = 'Tesla model s'
#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300
#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'en'

**Capturando os dados do twitter:**

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth)
#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

**Salvando os dados em uma planilha Excel:**

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Importando e limpando os tweets

Esta etapa é muito importante, porque a maioria dos posts acabam saindo com muita "sujeira", esta pode ser desde vírgulas até parênteses, que podem atrapalhar o classificador. Outra ferramenta de limpeza que usaremos será para manter somente o radical da palavra, assim vamos diminuir a variação de palavras com o mesmo radical mas complementos diferentes.

**Abrindo o arquivo:**<br>
Este já possui uma classificação feita manualmente.

In [3]:
#Atribuindo o arquivo com os tweets:
df = pd.read_excel('Tesla Model S.xlsx')

In [8]:
df_treinamento = df.drop('B1',axis=1)

...

#### Vamos criar uma função para a limpeza dos tweets:

Esta função será usada mais para frente.

In [12]:
#Vamos criar uma função que irá substituir os caracteres que trapalham o classificador:
def Cleaner (Tweets):
    #Os elementos na primeira posição do replace serão substituidos por espaços:
    Tweets = Tweets.replace("*","")
    Tweets = Tweets.replace("!","")
    Tweets = Tweets.replace("@","")
    Tweets = Tweets.replace("#","")
    Tweets = Tweets.replace("$","")
    Tweets = Tweets.replace("%","")
    Tweets = Tweets.replace("&","")
    Tweets = Tweets.replace("-","")
    Tweets = Tweets.replace("_","")
    Tweets = Tweets.replace("+","")
    Tweets = Tweets.replace("=","")
    Tweets = Tweets.replace("'","")
    Tweets = Tweets.replace("?","")
    Tweets = Tweets.replace(";","")
    Tweets = Tweets.replace(",","")
    Tweets = Tweets.replace(".","")
    Tweets = Tweets.replace(":","")
    Tweets = Tweets.replace(")","")
    Tweets = Tweets.replace("(","")
    Tweets = Tweets.replace("/","")
    Tweets = Tweets.replace('"',"")
    Tweets = Tweets.replace("[","")
    Tweets = Tweets.replace(']',"")
    Tweets = Tweets.replace("\ ","")
    
    #Aproveitando a nossa função ja vamos separar todas as palavras:
    Tweets = Tweets.split()

    return Tweets

In [ ]:
# COLOCAR NA DEF DE LIMPEZA: --------------------
ps = PorterStemmer()

def bla (tweets):
    
    post = word_tokenize(Tweet) 
    Tweets_stem = [ps.stem(word) for word in post]

In [ ]:
#Analisando quantos emails ham e spam existem no dataframe de treinamento:
Relevante, Irrelevante = (df_treinamento["B1"]).value_counts()
Total_treinamento = df_treinamento["B1"].value_counts().sum()
print("O dataframe de treinamento possui:\n","HAM: {0} emails\n".format(ham)
                                            ,"SPAM: {0} emails\n".format(spam),"\nTOTAL: {0}".format(ham+spam))

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

- https://www.geeksforgeeks.org/python-stemming-words-with-nltk/